<a href="https://colab.research.google.com/github/orhod/Basic-MineSweeper/blob/main/Logic/Admin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys,re


try:
    # Step 1: Clone the GitHub repository if not already present
    if not os.path.exists("/content/Project-Pheonix"):
        !git clone https://github.com/Cloud-Course-Group-Phoenix/Project-Pheonix.git /content/Project-Pheonix

    # Step 2: Change directory to project root
    %cd /content/Project-Pheonix

    # Step 3: Checkout the 'main' branch (or develop if you have one)
    !git fetch origin -q
    !git checkout main -q

    # Step 4: Add project directory to Python path
    sys.path.append("/content/Project-Pheonix/Logic")

    # Step 5: Install required Python packages (quietly)
    %pip install -q firebase
    %pip install -q importnb

    print("✅ Setup completed successfully.")
    from importnb import Notebook
    with Notebook():
        import CloudDB as dbService
        import Indexmqtt as indx

except Exception as e:
    print("❌ Setup failed:", str(e))

# Clear installation output for cleaner display
from IPython.display import clear_output
clear_output()

In [ ]:
# Get the top 10 most searched terms from the index
def get_top_search_terms():
    stats = dbService.get_from_db('indexStats')
    if not stats or 'search_counts' not in stats:
        return []
    sorted_terms = sorted(
        stats['search_counts'].items(),
        key=operator.itemgetter(1),
        reverse=True
    )[:10]
    if sorted_terms:
        return sorted_terms
    return []


# Record that a term was searched
def record_search_term(term):
    stats = dbService.get_from_db('indexStats')
    if not stats:
        stats = {"search_counts": {"placeholder": 0}}
    elif 'search_counts' not in stats:
        stats['search_counts'] = {"placeholder": 0}

    # Remove placeholder if it exists and this isn't a placeholder term
    if "placeholder" in stats['search_counts'] and term != "placeholder":
        if len(stats['search_counts']) > 1:  # Only remove if there are other terms
            del stats['search_counts']["placeholder"]

    # Increment search count for this term
    if term in stats['search_counts']:
        stats['search_counts'][term] += 1
    else:
        stats['search_counts'][term] = 1

    # Save back to DB
    dbService.insert_to_db_stats(stats)


# Get the current status of the index
def update_index_status():
    stats = dbService.get_from_db('indexStats')
    if not stats:
      dbService.init_stats_if_needed()
    return {
      "word_count": stats.get("word_count", 0),
      "page_count": stats.get("page_count", 0),
      "last_indexed": stats.get("last_indexed", "Unknown")
    }



def get_daily_tasks():
    return dbService.get_from_db('dailyTasks') or []

def add_daily_task(task):
    if not task:
        return
    tasks = get_daily_tasks()
    tasks.append(task)
    dbService.insert_to_db_task(tasks)

def remove_daily_task(task):
    tasks = get_daily_tasks()
    if task in tasks:
        tasks.remove(task)
        dbService.insert_to_db_task(tasks)



def get_index_status():
  return dbService.get_from_db('indexStats')

def reindex_content():
  return indx.index_mqtt_website()

# Track search terms for stats
def track_search_terms(query):
    if query :
        words = re.findall(r'\w+', query.lower())
        for word in words:
            indx.record_search_term(word)
"""
# Check if MQTT broker is connected
def get_mqtt_connection_status():
    # Get the singleton instance
    mqtt_conn = MqttConnection.get_instance()
    if mqtt_conn is not None:
        return mqtt_conn.connected
    return False


# Reconnect to the MQTT broker
def reconnect_mqtt():
    try:
        # Get the singleton instance
        mqtt_conn = MqttConnection.get_instance()

        if mqtt_conn is None:
            # Create a new connection if one doesn't exist
            mqtt_conn = MqttConnection.get_instance(db_url)
            success = mqtt_conn.mqtt_handler()
        else:
            # Stop the current loop if it's running
            mqtt_conn.client.loop_stop()
            # Create a fresh connection
            success = mqtt_conn.mqtt_handler()

        if success:
            return "✅ Connected to MQTT Broker"
        else:
            return "❌ Failed to reconnect to MQTT Broker"
    except Exception as e:
        return f"Error reconnecting to MQTT: {str(e)}"
  """


'\n# Check if MQTT broker is connected\ndef get_mqtt_connection_status():\n    # Get the singleton instance\n    mqtt_conn = MqttConnection.get_instance()\n    if mqtt_conn is not None:\n        return mqtt_conn.connected\n    return False\n\n\n# Reconnect to the MQTT broker\ndef reconnect_mqtt():\n    try:\n        # Get the singleton instance\n        mqtt_conn = MqttConnection.get_instance()\n\n        if mqtt_conn is None:\n            # Create a new connection if one doesn\'t exist\n            mqtt_conn = MqttConnection.get_instance(db_url)\n            success = mqtt_conn.mqtt_handler()\n        else:\n            # Stop the current loop if it\'s running\n            mqtt_conn.client.loop_stop()\n            # Create a fresh connection\n            success = mqtt_conn.mqtt_handler()\n\n        if success:\n            return "✅ Connected to MQTT Broker"\n        else:\n            return "❌ Failed to reconnect to MQTT Broker"\n    except Exception as e:\n        return f"Error r